In [ ]:
import sys
sys.path.append("../")

## Exploring Feit-Thompson data set with GamePad

You can use this Jupyter notebook to explore the Feit-Thompson data set extracted by TCoq. You will need to have successfully built TCoq, Mathcomponents, and Feit-Thompson, as well as have successfully generated a tactic tree pickle to use this notebook. In the first step, we can obtain some basic statistics of the data set.

In [ ]:
from gamepad.exp_tactr_stats import *

# Change data/feit-tactr.log to where you saved <tactr.log>
stats, unique = load_tactr_stats("../data/feit-tactr.log")
tactr_stats = TacTrStats(stats)
print("Unique constants:", unique['const'])
print("Unique inductive types:", unique['ind'])
print("Unique constructors:", unique['conid'])
_ = tactr_stats.descrip_tacs()
_ = tactr_stats.descrip_tacsts()
_ = tactr_stats.descrip_term()
_ = tactr_stats.descrip_deadend()
_ = tactr_stats.gather_have_info()

### Visualizing proof trees

We can begin by visualizing proof trees just to get a sense of what the raw data set looks like. Below, we have hardcoded an example file and lemma from the Feit-Thompson formalization. You can mouse over the resulting graphic to see each node (proof state, consisting of context and goal) and each edge (tactic invocation) of the proof tree.

In [ ]:
from gamepad.tactr_prep import *

# Change me to target file of interest
file = "../data/odd-order/BGsection1.v.dump"
# Change me to target lemma of interest
tgtlem = "coprime_cent_prod"

vis = Visualize(f_display=True, f_jupyter=True, f_verbose=False)
tactr = vis.visualize_lemma(file, tgtlem)

### Distribution of Coq AST nodes

The proof trees above displayed pretty-printed proof states, but underneath, GamePad represents these objects as appropriate Python data structures. Coq proof states are encoded in a term language, which expresses logical formulas, computations, and proofs uniformly in the same language. Below, we can see just what the distribution of ASTs in the Coq term language are used in the Feit-Thompson formalization.

In [ ]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

coqexp_hist = tactr_stats.coqexp_hist()
data = [go.Bar(x=[p[0] for p in coqexp_hist], y=[p[1] for p in coqexp_hist])]

layout = go.Layout(
    title='CoqExp Usage (all lemmas)',
    xaxis=dict(
        title='Coq Expression'
    ),
    yaxis=dict(
        title='Usage',
    ),
)
fig = go.Figure(data=data, layout=layout)
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot(fig, filename='coqexp-hist')

### Scaling ASTs

There are 1602 lemmas in the formalization. Combining this with the total sizes of the ASTs above, we see that proof states contain on the order of 10000 nodes. To scale machine-learning models to such large ASTs, GamePad represents the ASTs in a shared format. Note that it is standard to use a shared representation of ASTs to scale in modern compiler and language implementations. In the next three cells, we will see how three different representations of the ASTs affects the sizes of the average AST involved in a proof context and proof goal.
1. Static-Share: a fully-shared representation
2. Static-Full: an unshared representation
3. Dynamic-Full: an unshared representation where we additionally eliminate lambdas/lets

In [ ]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

hist_sh, maxsize = tactr_stats.coqexp_comp_p('static_sh_comp', f_avg=True, f_trunc=True)
print("Maxsize", maxsize)
data = [go.Bar(x=[k for k, v in hist_sh.items()], y=[v for k, v in hist_sh.items()])]

layout = go.Layout(
    title='CoqExp Computation Static-Share',
    xaxis=dict(
        title='Size of Term'
    ),
    yaxis=dict(
        title='Number of terms',
    ),
)
fig = go.Figure(data=data, layout=layout)
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot(fig, filename='coqexp-sh-hist')

In [ ]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

hist_sf, maxsize = tactr_stats.coqexp_comp_p('static_full_comp', f_avg=True, f_trunc=True)
print("Maxsize", maxsize)
data = [go.Bar(x=[k for k, v in hist_sf.items()], y=[v for k, v in hist_sf.items()])]

layout = go.Layout(
    title='CoqExp Computation Static-Full',
    xaxis=dict(
        title='Size'
    ),
    yaxis=dict(
        title='Number',
    ),
)
fig = go.Figure(data=data, layout=layout)
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot(fig, filename='coqexp-sf-hist')

In [ ]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

hist_nm, maxsize = tactr_stats.coqexp_comp_p('cbname_comp', f_avg=True, f_trunc=True)
print("Maxsize", maxsize)
data = [go.Bar(x=[k for k, v in hist_nm.items()], y=[v for k, v in hist_nm.items()])]

layout = go.Layout(
    title='CoqExp Computation Call-By-Name',
    xaxis=dict(
        title='Size of Term'
    ),
    yaxis=dict(
        title='Number of Terms',
    ),
)
fig = go.Figure(data=data, layout=layout)
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot(fig, filename='coqexp-nm-hist')

### Tactics

We can also visualize the tactics used.

In [ ]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

tactic_hist = tactr_stats.avg_hist(f_sort=False)
print(tactic_hist)
data = [go.Bar(x=[p[0] for p in tactic_hist], y=[p[1] for p in tactic_hist])]

layout = go.Layout(
    title='Tactic Usage (all lemmas)',
    xaxis=dict(
        title='Tactic'
    ),
    yaxis=dict(
        title='Usage',
    ),
)
fig = go.Figure(data=data, layout=layout)
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot(fig, filename='tactic-hist')

### Statistics on proof contexts and goals

Finally, we can plot some of the properties of the proof context and sizes of the goals involved as a function of the depth in the proof tree that the context/goal occurs in. DepthMode.AST_CTX computes the sizes in terms of the size of the ASTs involved whereas DepthMode.AST_CHAR computes the sizes in terms of characters.

In [ ]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

avg_depth_ctx_size = tactr_stats.avg_depth_size(DepthMode.AST_CTX).items()
data = [go.Bar(x=[p[0] for p in avg_depth_ctx_size], y=[p[1] for p in avg_depth_ctx_size])]

layout = go.Layout(
    title='Average Context Size vs. Depth in Tree',
    xaxis=dict(
        title='Depth (proof tree)'
    ),
    yaxis=dict(
        title='Size of context (char-length)',
    ),
)
fig = go.Figure(data=data, layout=layout)
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot(fig, filename='average-context-size')

In [ ]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

avg_depth_goal_size = tactr_stats.avg_depth_size(DepthMode.CHAR_GOAL).items()
data = [go.Bar(x=[p[0] for p in avg_depth_goal_size], y=[p[1] for p in avg_depth_goal_size])]

layout = go.Layout(
    title='Average Goal Size vs. Depth in Tree',
    xaxis=dict(
        title='Depth (proof tree)'
    ),
    yaxis=dict(
        title='Size of goal (char-length)',
    ),
)
fig = go.Figure(data=data, layout=layout)
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot(fig, filename='average-goal-size')

In [ ]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

avg_depth_astctx_size = tactr_stats.avg_depth_size(DepthMode.AST_CTX).items()
data = [go.Bar(x=[p[0] for p in avg_depth_astctx_size], y=[p[1] for p in avg_depth_astctx_size])]

layout = go.Layout(
    title='Average Context Size vs. Depth in Tree',
    xaxis=dict(
        title='Depth (proof tree)'
    ),
    yaxis=dict(
        title='Size of context (ast-size)',
    ),
)
fig = go.Figure(data=data, layout=layout)
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot(fig, filename='average-context-size')

In [ ]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

avg_depth_astgoal_size = tactr_stats.avg_depth_size(DepthMode.AST_GOAL).items()
data = [go.Bar(x=[p[0] for p in avg_depth_astgoal_size], y=[p[1] for p in avg_depth_astgoal_size])]

layout = go.Layout(
    title='Average Goal Size vs. Depth in Tree',
    xaxis=dict(
        title='Depth (proof tree)'
    ),
    yaxis=dict(
        title='Size of context (ast-size)',
    ),
)
fig = go.Figure(data=data, layout=layout)
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot(fig, filename='average-context-size')